In [2]:
import pandas as pd
import numpy as np

df=pd.read_csv('Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X=df.drop(['RowNumber','CustomerId','Surname','Exited'],axis=1)
y=df['Exited']

In [5]:
X['Gender']=pd.get_dummies(X['Gender'],drop_first=True)

In [6]:
X['Geography']=X['Geography'].map(X['Geography'].value_counts().to_dict())

In [7]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,5014,0,42,2,0.00,1,1,1,101348.88
1,608,2477,0,41,1,83807.86,1,0,1,112542.58
2,502,5014,0,42,8,159660.80,3,1,0,113931.57
3,699,5014,0,39,1,0.00,2,0,0,93826.63
4,850,2477,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,5014,1,39,5,0.00,2,1,0,96270.64
9996,516,5014,1,35,10,57369.61,1,1,1,101699.77
9997,709,5014,0,36,7,0.00,1,0,1,42085.58
9998,772,2509,1,42,3,75075.31,2,1,0,92888.52


In [8]:
X.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [11]:
X_train

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
8371,813,2509,0,29,5,106059.40,1,0,0,187976.88
5027,681,2509,1,37,8,73179.34,2,1,1,25292.53
9234,642,2509,1,49,4,120688.61,1,1,0,24770.22
3944,612,5014,0,31,8,117989.76,1,1,1,54129.86
6862,586,5014,1,33,9,0.00,1,1,0,6975.02
...,...,...,...,...,...,...,...,...,...,...
5734,768,5014,1,54,8,69712.74,1,1,1,69381.05
5191,682,5014,0,58,1,0.00,1,1,1,706.50
5390,735,5014,0,38,1,0.00,3,0,0,92220.12
860,667,5014,1,43,8,190227.46,1,1,0,97508.04


In [12]:
X_train=torch.FloatTensor(X_train.values)
X_test=torch.FloatTensor(X_test.values)

In [13]:
y_train=torch.LongTensor(y_train.values)
y_test=torch.LongTensor(y_test.values)

In [34]:
y_train

tensor([1, 0, 1,  ..., 1, 1, 0])

In [14]:
class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.f1=nn.Linear(10,20)
        self.f2=nn.Linear(20,15)
        self.out=nn.Linear(15,2)
    def forward(self,x):
        x=F.relu(self.f1(x))
        x=F.relu(self.f2(x))
        x=self.out(x)
        return x

In [15]:
'''model=nn.Sequential(nn.Linear(10,20),
                   nn.Linear(20,15),
                   nn.Linear(15,2),
                   nn.Sigmoid())'''

'model=nn.Sequential(nn.Linear(10,20),\n                   nn.Linear(20,15),\n                   nn.Linear(15,2),\n                   nn.Sigmoid())'

In [16]:
model=model()
model.parameters

<bound method Module.parameters of model(
  (f1): Linear(in_features=10, out_features=20, bias=True)
  (f2): Linear(in_features=20, out_features=15, bias=True)
  (out): Linear(in_features=15, out_features=2, bias=True)
)>

In [17]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters())

In [25]:
epochs=1500
final_losses=[]
for epoch in range(epochs):
    epoch+=1
    y_pred=model(X_train)
    loss=criterion(y_pred,y_train)
    final_losses.append(loss)
    if epoch%10==1:
        print("epoch number is",epoch," and loss function is ",loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

epoch number is 1  and loss function is  5.240283966064453
epoch number is 11  and loss function is  5.6597161293029785
epoch number is 21  and loss function is  2.336468458175659
epoch number is 31  and loss function is  3.0074195861816406
epoch number is 41  and loss function is  2.933842182159424
epoch number is 51  and loss function is  3.7080132961273193
epoch number is 61  and loss function is  2.2447140216827393
epoch number is 71  and loss function is  5.34783411026001
epoch number is 81  and loss function is  4.201772212982178
epoch number is 91  and loss function is  3.365476369857788
epoch number is 101  and loss function is  4.898653984069824
epoch number is 111  and loss function is  6.930049419403076
epoch number is 121  and loss function is  2.0335521697998047
epoch number is 131  and loss function is  6.27794885635376
epoch number is 141  and loss function is  4.855041027069092
epoch number is 151  and loss function is  3.910621404647827
epoch number is 161  and loss fu

In [27]:
predictions_test=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        predictions_test.append(model((data)).argmax().item())
        print(model(data).argmax().item())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [28]:
predictions_train=[]
with torch.no_grad():
    for i,data in enumerate(X_train):
        predictions_train.append(model((data)).argmax().item())
        print(model(data).argmax().item())

Streaming output truncated to the last 5000 lines.
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

In [29]:
from sklearn.metrics import accuracy_score
accuracy_test=accuracy_score(predictions_test,y_test)
accuracy_train=accuracy_score(predictions_train,y_train)
print("Accuracy of test set : ",accuracy_test)
print("Accuracy of train set : ",accuracy_train)


Accuracy of test set :  0.8054545454545454
Accuracy of train set :  0.7926865671641791


In [30]:
from sklearn.metrics import confusion_matrix
confusion_matrix(predictions,y_test)

array([[2657,  642],
       [   0,    1]])

In [2]:
import torch
n=torch.rand(16,2,2,1)
n

tensor([[[[0.7500],
          [0.6417]],

         [[0.7798],
          [0.8141]]],


        [[[0.3505],
          [0.0069]],

         [[0.7444],
          [0.2675]]],


        [[[0.6101],
          [0.1771]],

         [[0.5371],
          [0.4149]]],


        [[[0.1736],
          [0.5624]],

         [[0.5754],
          [0.8863]]],


        [[[0.1953],
          [0.7826]],

         [[0.6282],
          [0.5717]]],


        [[[0.4998],
          [0.8413]],

         [[0.0109],
          [0.3343]]],


        [[[0.7287],
          [0.3771]],

         [[0.8241],
          [0.9286]]],


        [[[0.1604],
          [0.9383]],

         [[0.5929],
          [0.4502]]],


        [[[0.2179],
          [0.0909]],

         [[0.9562],
          [0.3799]]],


        [[[0.2200],
          [0.1353]],

         [[0.9870],
          [0.2607]]],


        [[[0.3933],
          [0.3142]],

         [[0.0758],
          [0.2410]]],


        [[[0.8866],
          [0.8866]],

         [[0